In [29]:
from Bio import AlignIO
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.plotly as py
init_notebook_mode(connected=True)

import pandas as pd

In [ ]:
"""simgen

This script allows the user to create similarity plots which are useful 
to analyze possible recombination events in DNA sequences. 

The simgen function accepts alignment file in fasta format

The output of simgen is plotly based similarity plot which is easy to manipulate.

This script requires that 'pandas', 'biopython' and 'plotly' by installed within the Python 
environment your are running this script in

"""

In [ ]:
#TODO: pdistance and draw_simplot should be private(don't forget to correct their calls inside simgen)

In [25]:
def pdistance(seq1, seq2):
    """calculates pairwise distance between two sequences"""
    p = 0
    pairs = []
    for x in zip(seq1,seq2):
        if '-' not in x: pairs.append(x)
    for (x,y) in pairs:
        if x != y:
            p += 1
    length = len(pairs)
    
    return float(1 - p / length) # '1 minus the distance' is just for taking 'upside down' the plot for nice view

In [26]:
def draw_simplot(distance_data, tick_container):
    """draws similarity plot"""
    
    data = []    

    for key in distance_data.keys():
        trace = go.Scatter(y=distance_data[key], x=tick_container, name=key)
        data.append(trace)
    
    layout = go.Layout(
        title="similarity plot",
        xaxis=dict(
            title="nucleotide position"),
        yaxis=dict(
            title="sequence identity"),
        legend=dict(x=-0.1, y=1.5))
    
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)
 
    

In [34]:
def simgen(align, pot_rec, window=500, shift=100, region=False, return_data=False):
    """slices the alignment, collects the distance data
    
    Parameters:
    -----------
    align: the location of the fasta alignment
    pot_rec: int
        the number of the sequence under study, starts with 0, 
        like the 'x' dimension in the numpy array 
    window: int
        sliding window size
    shift: int
        the step window slides downstream the alignment
    region: a tuple or a list of two integers
        the region of the alignment to analyze. the start and the end 
        nucleotide positions
    return_data: bool, optional
        return the data in pandas DataFrame 
    """
        
    if region:
        align = AlignIO.read(align, "fasta")
        align = align[:, region[0] : region[1]]
        left_border = region[0]   # border for the first tick
    else:
        align = AlignIO.read(align, "fasta")
        left_border = 1  # border for the first tick
        
    distance_data = {}
    parents = list(range(0, len(align)))
    parents.remove(pot_rec)
    align_length = len(align[0, :])
    
    # creating tick labels for the plot
    tick_container = []      
    tick_container.append(left_border)
    while tick_container[-1] < align_length:
        tick_container.append(tick_container[-1] + shift)
    
    for par in parents:
        dist_container = []
        start = 0
        finish = shift

        while start < align_length:
            seq1 = align[pot_rec, start:finish].seq # here is a potential recombinant sequence slice
            seq2 = align[par, start:finish].seq  # here's a parent's slice
            dist_container.append(pdistance(seq1, seq2)) #calculate pdistance, append to container
            start += shift
            finish = start + window

        distance_data[align[par].id] = dist_container
    
    if return_data:
        data = pd.DataFrame(data=distance_data, index=tick_container[1:]) # [1:] to map data to index
        return data
    else:
        draw_simplot(distance_data, tick_container) 
    


In [36]:
data_df = simgen("for_recan_lsdv.fasta", window=500, shift=100, pot_rec=2, return_data=True)

In [37]:
data_df.head()

,AF409138.1_Lumpy_skin_disease_virus_iso,mapping_consensus_on_AF409138.1_default,NODE_1_length_146178_cov_19.4988,contig_assembly_2
101,0.990,0.990,0.990,0.990
201,0.996,0.996,0.996,0.996
301,0.996,0.996,0.996,0.996
401,0.992,0.992,0.992,0.992
501,0.990,0.990,0.990,0.990


In [28]:
simgen("for_recan_lsdv.fasta", window=500, shift=100, pot_rec=2)

In [ ]:
data = simgen("for_recan_lsdv.fasta", window=500, shift=100, pot_rec=2, save_data=True)

In [ ]:
simgen("for_recan_lsdv.fasta", window=500, shift=100, pot_rec=2, region=(1, 15000))

In [18]:
simgen("for_recan_lsdv.fasta", window=500, shift=100, pot_rec=2, region=(120000, 150000))

In [9]:
simgen("for_recan_lsdv_1.fasta", window=500, shift=100, pot_rec=2)

In [8]:
simgen("for_recan_lsdv_1.fasta", window=500, shift=100, pot_rec=2, region=(120000, 150000))